# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpgrowth, fpmax, association_rules

In [2]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


### 1. Выбрать данные по странам Germany, Netherlands (совместно): <br>

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289955 entries, 0 to 289954
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user     289955 non-null  int64 
 1   artist   289955 non-null  object
 2   sex      289955 non-null  object
 3   country  289955 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.8+ MB


In [4]:
source = data[data.country == ('Germany')]
source = source.append(data[data.country == ('Netherlands')], ignore_index=True)

source

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany
...,...,...,...,...
33919,19685,the cure,m,Netherlands
33920,19685,alice in chains,m,Netherlands
33921,19685,haggard,m,Netherlands
33922,19685,system of a down,m,Netherlands



### 2. Поиск ассоциативных правил</br>
#### Предобработка данных</br>

In [5]:
source['user'].unique()

array([    1,    33,    42, ..., 19476, 19681, 19685], dtype=int64)

In [6]:
source['artist'].unique()

array(['red hot chili peppers', 'the black dahlia murder', 'goldfrapp',
       ..., 'primal scream', 'los hermanos', 'the jesus and mary chain'],
      dtype=object)

In [7]:
source['sex'].unique()

array(['f', 'm'], dtype=object)

In [8]:
source['country'].unique()

array(['Germany', 'Netherlands'], dtype=object)

In [9]:
source = source[(source['artist'] != '[unknown]')]
source.count()

user       33829
artist     33829
sex        33829
country    33829
dtype: int64

In [10]:
source = source.drop_duplicates()
source.count()

user       33828
artist     33828
sex        33828
country    33828
dtype: int64

#### 2.1. Apriori </br>

In [11]:
source_by_artist = source.groupby(['user', 'sex', 'country'])['artist'].apply(','.join).reset_index()
source_by_artist

,user,sex,country,artist
0,1,f,Germany,"red hot chili peppers,the black dahlia murder,..."
1,31,f,Netherlands,"jamiroquai,tori amos,jazzanova,radiohead,st. g..."
2,33,f,Germany,"death cab for cutie,tegan and sara,kimya dawso..."
3,42,f,Germany,"soundtrack,groove coverage,avril lavigne,the r..."
4,51,f,Germany,"kate nash,arctic monkeys,lykke li,adele,tegan ..."
...,...,...,...,...
1718,19662,m,Germany,"blind guardian,in extremo,subway to sally,finn..."
1719,19672,m,Germany,"system of a down,björk,portishead,the prodigy,..."
1720,19681,m,Netherlands,"the strokes,muse,led zeppelin,snow patrol,radi..."
1721,19685,m,Netherlands,"philip glass,rammstein,pink floyd,moby,blank &..."


In [12]:
## Приводим таблицу в бинарный вид
dummy_source_by_artist = source_by_artist['artist'].str.get_dummies(',')
dummy_source_by_artist

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,a day to remember,...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1718,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1719,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1720,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1721,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
frequent = apriori(dummy_source_by_artist, min_support=0.02, use_colnames=True)
rules = association_rules(frequent, metric="confidence", min_threshold=0.2)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(3 doors down),(linkin park),0.057458,0.142194,0.022635,0.393939,2.770439,0.014465,1.415380
1,(ac/dc),(die Ärzte),0.065003,0.143355,0.020894,0.321429,2.242192,0.011575,1.262425
2,(ac/dc),(metallica),0.065003,0.110273,0.020313,0.312500,2.833882,0.013145,1.294149
3,(ac/dc),(red hot chili peppers),0.065003,0.154962,0.023215,0.357143,2.304708,0.013142,1.314503
4,(air),(coldplay),0.087057,0.171793,0.023796,0.273333,1.591059,0.008840,1.139734
...,...,...,...,...,...,...,...,...,...
304,(system of a down),(the offspring),0.120720,0.068485,0.028439,0.235577,3.439822,0.020171,1.218585
305,(the doors),(the beatles),0.047591,0.123622,0.023796,0.500000,4.044601,0.017912,1.752757
306,(the rolling stones),(the beatles),0.048172,0.123622,0.022055,0.457831,3.703490,0.016099,1.616431
307,(the killers),(the kooks),0.105049,0.094602,0.034243,0.325967,3.445650,0.024305,1.343254


#### Правило 1
lift > 1 => Правило сильное => Группы, указанные консеквентами, можно рекомендовать слушателям группы linkin park</br>

In [14]:
rule1 = rules[(rules['antecedents'] == {'linkin park'}) ].sort_values(by = ['lift'], ascending=False)
rule1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
206,(linkin park),(koЯn),0.142194,0.069066,0.029019,0.204082,2.954896,0.019198,1.169636
55,(linkin park),(billy talent),0.142194,0.083575,0.033082,0.232653,2.783759,0.021198,1.194277
167,(linkin park),(foo fighters),0.142194,0.094602,0.028439,0.200000,2.114110,0.014987,1.131747
226,(linkin park),(system of a down),0.142194,0.120720,0.033082,0.232653,1.927217,0.015916,1.145871
136,(linkin park),(die Ärzte),0.142194,0.143355,0.037145,0.261224,1.822226,0.016760,1.159548
222,(linkin park),(red hot chili peppers),0.142194,0.154962,0.040046,0.281633,1.817427,0.018012,1.176331
219,(linkin park),(rammstein),0.142194,0.132327,0.031921,0.224490,1.696473,0.013105,1.118841
95,(linkin park),(coldplay),0.142194,0.171793,0.033082,0.232653,1.354261,0.008654,1.079312


#### Правило 2
lift < 1 => Правило рекомендовать не стоит => Слушателям группы rammstein нельзя рекомендовать группу coldplay

In [15]:
rules2 = association_rules(frequent, metric="confidence", min_threshold=0.15)

In [16]:
rule2 = rules2[(rules2['lift'] < 1)].sort_values(by = ['lift'])
rule2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
117,(rammstein),(coldplay),0.132327,0.171793,0.020313,0.153509,0.893566,-0.00242,0.9784


#### 2.2. FP-growth </br>

In [17]:
dummy_source_by_artist_sex = pd.concat([source_by_artist['sex'].str.get_dummies(','),
                             source_by_artist['artist'].str.get_dummies(',')], axis = 1)

In [18]:
frequent_fp = fpgrowth(dummy_source_by_artist_sex, min_support=0.02, use_colnames=True)

In [19]:
rules_fp = association_rules(frequent_fp, metric="confidence", min_threshold=0.2)
rules_fp

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(red hot chili peppers),(f),0.154962,0.233894,0.043529,0.280899,1.200965,0.007284,1.065366
1,(red hot chili peppers),(m),0.154962,0.766106,0.111434,0.719101,0.938645,-0.007284,0.832664
2,(red hot chili peppers),(coldplay),0.154962,0.171793,0.047011,0.303371,1.765905,0.020390,1.188877
3,(coldplay),(red hot chili peppers),0.171793,0.154962,0.047011,0.273649,1.765905,0.020390,1.163401
4,"(red hot chili peppers, coldplay)",(m),0.047011,0.766106,0.029019,0.617284,0.805743,-0.006996,0.611143
...,...,...,...,...,...,...,...,...,...
775,(deep purple),(m),0.025537,0.766106,0.023215,0.909091,1.186639,0.003651,2.572838
776,(frank sinatra),(m),0.034243,0.766106,0.027858,0.813559,1.061941,0.001625,1.254524
777,(ensiferum),(m),0.035403,0.766106,0.028439,0.803279,1.048522,0.001316,1.188963
778,(katie melua),(m),0.027858,0.766106,0.020313,0.729167,0.951783,-0.001029,0.863610


#### Правило 3
Высокие значения параметров confidence, lift и conviction говорят о том, что следующие группы стоит рекомендовать девушкам </br>

In [20]:
rule3 = rules_fp[(rules_fp['lift'] > 2) 
                 & (rules_fp['consequents'] == {'f'})
                 & (rules_fp['confidence'] > 0.5)
                 & (rules_fp['conviction'] > 1.5)].sort_values(by = ['confidence'], ascending=False)
rule3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
207,(paramore),(f),0.037145,0.233894,0.021474,0.578125,2.471735,0.012786,1.815954
464,(pink),(f),0.037725,0.233894,0.021474,0.569231,2.433709,0.012651,1.778459
296,(kate nash),(f),0.040627,0.233894,0.022055,0.542857,2.320950,0.012552,1.675856


In [21]:
dummy_source_by_artists_country = pd.concat([source_by_artist['country'].str.get_dummies(','),
                             source_by_artist['artist'].str.get_dummies(',')], axis = 1)

In [22]:
frequent_fp2 = fpgrowth(dummy_source_by_artists_country, min_support=0.02, use_colnames=True)
rules_fp2 = association_rules(frequent_fp2, metric="confidence", min_threshold=0.2)
rules_fp2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(red hot chili peppers),(Germany),0.154962,0.728961,0.107371,0.692884,0.950509,-0.005591,0.882529
1,(red hot chili peppers),(Netherlands),0.154962,0.271039,0.047591,0.307116,1.133107,0.005591,1.052068
2,(red hot chili peppers),(coldplay),0.154962,0.171793,0.047011,0.303371,1.765905,0.020390,1.188877
3,(coldplay),(red hot chili peppers),0.171793,0.154962,0.047011,0.273649,1.765905,0.020390,1.163401
4,"(red hot chili peppers, coldplay)",(Germany),0.047011,0.728961,0.033662,0.716049,0.982287,-0.000607,0.954528
...,...,...,...,...,...,...,...,...,...
970,(editors),(Netherlands),0.042368,0.271039,0.021474,0.506849,1.870024,0.009991,1.478171
971,(black sabbath),(Germany),0.026698,0.728961,0.022635,0.847826,1.163061,0.003173,1.781113
972,(james blunt),(Germany),0.026698,0.728961,0.021474,0.804348,1.103417,0.002013,1.385310
973,(ensiferum),(Germany),0.035403,0.728961,0.031341,0.885246,1.214394,0.005533,2.361910


In [23]:
rules_fp2["antecedent_len"] = rules_fp2["antecedents"].apply(lambda x: len(x))

#### Правило 4
lift > 9 => </br>
Слушателям sсhandmaul из Германии можно рекомендовать subway to sally</br>
Слушателям subway to sally из Германии можно рекомендовать schandmaul</br>
Слушателям in extremo из Германии можно рекомендовать sсhandmaul</br>
Слушателям in extremo из Германии можно рекомендовать subway to sally</br>

In [24]:
rule4 = rules_fp2[(rules_fp2['antecedent_len'] > 1) & (rules_fp2['lift'] > 9)]
rule4

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
52,"(subway to sally, Germany)",(schandmaul),0.055136,0.044689,0.023215,0.421053,9.421736,0.020751,1.650082,2,1
54,"(Germany, schandmaul)",(subway to sally),0.043529,0.055717,0.023215,0.533333,9.572222,0.020790,2.023464,2,1
57,"(in extremo, Germany)",(schandmaul),0.049913,0.044689,0.020313,0.406977,9.106765,0.018083,1.610916,2,1
325,"(in extremo, Germany)",(subway to sally),0.049913,0.055717,0.026117,0.523256,9.391352,0.023336,1.980692,2,1


#### Правило 5
lift > 1
Слушателям группы pink floyd можно рекомендовать группы Netherlands и The beatles

In [29]:
rule5 = rules_fp2[(rules_fp2['antecedents'] == {'pink floyd'}) & (rules_fp2['lift'] > 1)]
rule5

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
959,(pink floyd),(Netherlands),0.06036,0.271039,0.024956,0.413462,1.525469,0.008597,1.242819,1,1
960,(pink floyd),(the beatles),0.06036,0.123622,0.020313,0.336538,2.722328,0.012852,1.320918,1,1
